# Computational Experiments

This notebook implements the three pricing policies (static pricing, iterative pricing, and match-based pricing) on the Chicago dataset assuming a cost $c=0.9$, an average sojourn time of 3 min, a sharing disutility factor $\beta_0=0.05$, and a detour disutility factor $\beta_0=0.675$. Each policy is trained and evaluated on both training and test datasets. Detailed experimental methodology can be found in Section 4 of the paper, and the complete evaluation results are presented in Table 2 in Section 4 and Table EC.2 in Appendix EC.3.3.

## 1. Setup and Dependencies

In [ ]:
# external packages
import numpy as np
import copy
from gurobipy import *
from collections import defaultdict
import pandas as pd
import pickle
from IPython.display import display
import os
import sys

# add current directory to path
sys.path.append(os.getcwd())

# internal packages
from lib.network import osm
from lib.instance_data import InstanceData
from lib.utils import (
    import_demand,
    TRAINING_SET_KEY,  # training dataset key
    TEST_SET_KEY,  # test dataset key
    MATCH_KEY,  # match-based pricing policy key
    STATIC_KEY,  # static pricing policy key
    ITERATIVE_KEY,  # iterative pricing policy key
    SYN_DATA_KEY,  # synthetic data key
    REAL_DATA_KEY,  # real data key
)
from lib.policies import (
    MatchPricingAffineMatchingCombined,
    StaticPricingAffineMatchingCombined,
    IterativePricingAffineMatchingCombined,
    StaticPricingPolicy,
    MatchBasedPricingPolicy,
    AffineMatchingPolicy,
)
from lib.simulation import Simulator
from lib.evaluator import evaluator

## 2. Data Loading

In [ ]:
# import the Chicago network data, and calculate the shortest times between nodes
with open("data/Chicago_network.pickle", "rb") as f:
    network_data = pickle.load(f)
network = osm(network_data["G"], network_data["edge_keys"])
network.calculate_shortest_times()

# import the demand data
rider_types, arrival_rates, arrival_types, arrival_times = import_demand(
    "data/Chicago_demand.pickle"
)
n_rider_types = len(rider_types)

## 3. Experiment Settings

Set the parameters for the experiments, including the paramters for policy optimization, simulation, and evaluation. For each policy, two optimization parameter sets are established: one with $\beta_0=0, \beta_1=0$, and another with $\beta_0=0.05, \beta_1=0.675$; the optimized affine weights of static and match-based pricing under the former setting will be used as a warmup for the latter.

In [ ]:
c = 0.9  # cost from 0 to 2 (in dollars per mile)
sojourn_time = 180  # average sojourn time (in seconds)

# ------------- Optimization parameters ------------
# Define common parameters
BASE_CONFIG = {
    STATIC_KEY: {
        "eps_fluid": 1e-8,  # termination criterion for optimizing the fluid model
        "bootstrap_rounds": 2,  # number of bootstrap rounds for states sampling
        "alpha_bound": 1,  # bounding alpha values by c*solo_length (solo dispatching cost)
        "shrinkage_factor": 1,  # shrinkage factor for updating the disutility
        "momentum": 0.8,  # how much to weigh the new disutility when updating the disutility
    },
    ITERATIVE_KEY: {
        "bootstrap_rounds": 1,
        "alpha_bound": 1,
        "shrinkage_factor": 1,
        "momentum": 0.1,
        "iter_max_iter": 10,  # maximum number of iterations for iterative pricing policy
    },
    MATCH_KEY: {
        "bootstrap_rounds": 2,
        "alpha_bound": 1,
        "momentum": 0.8,
        "shrinkage_factor": 1,
        "epsilon_cutting_plane": 0.5,  # termination criterion for cutting plane
        "early_stop": True,  # stop cutting plane optimization when the objective value is not improved
    },
}
# maximum detour rate
MAX_DETOUR_RATE = 0.25

opt_params = dict()
# parameters when beta0=0, beta1=0
# policies have no warmup affine weights as the first policy for states sampling
opt_params[0, 0] = {
    STATIC_KEY: {
        **BASE_CONFIG[STATIC_KEY],
        **{
            "warmup": 0,
            "warmup_weights": None,
            "update_disutil_opt": False,  # whether to update the disutility in the optimization
            "update_disutil_eval": False,  # whether to update the disutility in the evaluation
        },
    },
    MATCH_KEY: {
        **BASE_CONFIG[MATCH_KEY],
        **{
            "warmup": 0,
            "warmup_weights": None,
            "update_disutil_opt": False,
            "update_disutil_eval": False,
        },
    }
}

# parameters when beta0=0.05, beta1=0.675
# static and match-based policies use the affine weights in the results of beta0=beta1=0 as the warmup policy for states sampling
opt_params[0.05, 0.675] = {
    STATIC_KEY: {
        **BASE_CONFIG[STATIC_KEY],
        **{
            "warmup": 1,
            "update_disutil_opt": True,
            "update_disutil_eval": True,
        },
    },
    MATCH_KEY: {
        **BASE_CONFIG[MATCH_KEY],
        **{"warmup": 1, "update_disutil_opt": True, "update_disutil_eval": True},
    },
    ITERATIVE_KEY: {
        **BASE_CONFIG[ITERATIVE_KEY],
        **{"update_disutil_opt": True, "update_disutil_eval": True},
    },
}

# ------------- Simulation parameters ------------
sim_params = {
    # week numbers for training/validation/test data sets
    "data_sets_division": {
        REAL_DATA_KEY: [7, 0, 1],  # real data
        SYN_DATA_KEY: [1, 0, 0],  # synthetic data
    },
    # simulation time limits for training and test data sets
    "time_limits": {
        TRAINING_SET_KEY: 100000,
        TEST_SET_KEY: 3600,
    },
    # simulation time for greedy matching policy
    "time_limit_greedy": 40000,
    # the observing rate for constraint sampling, which is a portion rate of the summation of total arrival rates and reneging rates
    "observer_rate_portion": 0.1,
    # a sufficiently long time for evaluating the synthetic training set during optimization
    "evaluation_time": 1000000,
}

# ------------- Evaluation parameters ------------
# maximum number of iterations (of updating the disutility) for evaluation
EVAL_MAX_ITER = 10
# evaluation time for synthetic training set
training_synthetic_eval_time = 2000000
# number of repetitions for test set evaluation
n_repeats = 100

## 4. Policy training

Train the static, iterative, and match-based pricing policies using a synthetic training dataset generated based on arrival rates estimated from the actual training data. The synthetic dataset is extended to be sufficiently long to ensure accurate estimation of disutilities and performance metrics, as well as diversifying the sampled states. We first train the policies for $\beta_0=\beta_1=0$; the affine weights from the static and match-based pricing policies are used as the warmup weights for subsequent training of the static and match-based pricing policies with $\beta_0=0.05, \beta_1=0.675$.

In [ ]:
# static pricing
static_prices = dict()
static_weights = dict()
static_disutilities = dict()
fluid_profits = dict()
# iterative pricing
iterative_prices = dict()
iterative_weights = dict()
iterative_disutilities = dict()
# match-based pricing
match_weights = dict()
match_active_disutilities = dict()
match_passive_disutilities = dict()
# training time
training_time = defaultdict(defaultdict)

# train the pricing policies
for beta0, beta1 in opt_params.keys():

    # ================== train the static pricing policy ==================
    SPAMC = StaticPricingAffineMatchingCombined()
    (
        fluid_static_prices,
        fluid_disutility,
        fluid_active_disutility,
        fluid_passive_disutility,
        fluid_profit,
        opt_static_weights,
        opt_disutility,
        static_time,
    ) = SPAMC.train_static_pricing_policy(
        c,
        sojourn_time,
        network,
        rider_types,
        arrival_rates,
        arrival_types,
        arrival_times,
        beta0,
        beta1,
        MAX_DETOUR_RATE,
        opt_params[beta0, beta1][STATIC_KEY],
        sim_params,
        verbose=False,
    )
    # save the optimal static prices and weights
    static_prices[beta0, beta1] = fluid_static_prices
    static_weights[beta0, beta1] = opt_static_weights
    static_disutilities[beta0, beta1] = opt_disutility
    fluid_profits[beta0, beta1] = fluid_profit
    training_time[beta0, beta1][STATIC_KEY] = static_time
    print(f"========= beta0={beta0}, beta1={beta1} =========")
    print(f"Static pricing policy was trained in {static_time:.2f} seconds.")

    # ================== train the iterative pricing policy ==================
    if beta0 > 0 or beta1 > 0:
        IPAMC = IterativePricingAffineMatchingCombined()
        (
            opt_iterative_price,
            opt_iterative_weights,
            iterative_opt_disutility,
            iterative_time,
        ) = IPAMC.train_iterative_pricing_policy(
            c,
            sojourn_time,
            network,
            rider_types,
            arrival_rates,
            arrival_types,
            arrival_times,
            beta0,
            beta1,
            MAX_DETOUR_RATE,
            opt_params[beta0, beta1][ITERATIVE_KEY],
            sim_params,
            verbose=False,
        )
        # save the optimal iterative price and weights
        iterative_prices[beta0, beta1] = opt_iterative_price
        iterative_weights[beta0, beta1] = opt_iterative_weights
        iterative_disutilities[beta0, beta1] = iterative_opt_disutility
        training_time[beta0, beta1][ITERATIVE_KEY] = iterative_time
        print(f"Iterative pricing policy was trained in {iterative_time:.2f} seconds.")

    # ================== train the match-based pricing policy ==================
    MPAMC = MatchPricingAffineMatchingCombined()
    (
        match_affine_weights,
        match_active_disutility,
        match_passive_disutility,
        match_time,
    ) = MPAMC.train_matchbased_pricing_policy(
        c,
        sojourn_time,
        network,
        rider_types,
        arrival_rates,
        arrival_types,
        arrival_times,
        beta0,
        beta1,
        MAX_DETOUR_RATE,
        opt_params[beta0, beta1][MATCH_KEY],
        sim_params,
        fluid_disutility,
        fluid_active_disutility,
        fluid_passive_disutility,
        verbose=False,
    )
    # save the optimal match-based weights
    match_weights[beta0, beta1] = match_affine_weights
    match_active_disutilities[beta0, beta1] = match_active_disutility
    match_passive_disutilities[beta0, beta1] = match_passive_disutility
    training_time[beta0, beta1][MATCH_KEY] = match_time
    print(f"Match-based pricing policy was trained in {match_time:.2f} seconds.")

    # set the warmup weights for beta0=0.05, beta1=0.675
    if beta0 == 0 and beta1 == 0:
        opt_params[0.05, 0.675][STATIC_KEY]["warmup_weights"] = static_weights[
            beta0, beta1
        ]
        opt_params[0.05, 0.675][MATCH_KEY]["warmup_weights"] = match_weights[
            beta0, beta1
        ]

# =================== save the training results ===================
# create the pricing and matching policies based on the training results
beta0, beta1 = 0.05, 0.675
pricing_policy = dict()
matching_policy = dict()

instance_data = InstanceData(
    c,
    sojourn_time,
    network,
    rider_types,
    arrival_rates,
    arrival_types,
    arrival_times,
    beta0,
    beta1,
    MAX_DETOUR_RATE,
)

# create the static pricing policy
static_instance = copy.deepcopy(instance_data)
static_instance.disutility = static_disutilities[beta0, beta1]
pricing_policy[STATIC_KEY] = StaticPricingPolicy(
    static_instance, static_prices[beta0, beta1]
)
matching_policy[STATIC_KEY] = AffineMatchingPolicy(
    static_instance,
    static_weights[beta0, beta1],
    match_flag=pricing_policy[STATIC_KEY].match_flag,
)

# create the iterative pricing policy
iterative_instance = copy.deepcopy(instance_data)
iterative_instance.disutility = iterative_disutilities[beta0, beta1]
pricing_policy[ITERATIVE_KEY] = StaticPricingPolicy(
    iterative_instance, iterative_prices[beta0, beta1]
)
matching_policy[ITERATIVE_KEY] = AffineMatchingPolicy(
    iterative_instance,
    iterative_weights[beta0, beta1],
    match_flag=pricing_policy[ITERATIVE_KEY].match_flag,
)

# create the match-based pricing policy
match_instance = copy.deepcopy(instance_data)
match_instance.active_disutility = match_active_disutilities[beta0, beta1]
match_instance.passive_disutility = match_passive_disutilities[beta0, beta1]
pricing_policy[MATCH_KEY] = MatchBasedPricingPolicy(
    match_instance, match_weights[beta0, beta1]
)
matching_policy[MATCH_KEY] = AffineMatchingPolicy(
    match_instance,
    match_weights[beta0, beta1],
    match_flag=pricing_policy[MATCH_KEY].match_flag,
)

========= beta0=0, beta1=0 =========
Static pricing policy was trained in 757.36 seconds.
Match-based pricing policy was trained in 11678.66 seconds.
========= beta0=0.05, beta1=0.675 =========
Static pricing policy was trained in 2221.87 seconds.
Iterative pricing policy was trained in 8099.88 seconds.
Match-based pricing policy was trained in 4763.62 seconds.


## 5. Policy evaluation

Evaluate the trained policies on both the training and test datasets.
- **Performance metrics:** include the average profit (\\$/min), average quoted price (\\$/mile $\cdot$ rider), average quoted disutility (\\$/mile $\cdot$ rider), average payment (\\$/mile $\cdot$ request), average realized disutility (\\$/mile $\cdot$ request), throughput (requests/min), match rate, cost efficiency, and average detour rate (over matched requests). Please refer to Section 4 of the paper for the detailed explanations of these metrics. 
- **Evaluation methods:**
    - *Training data evaluation:* Performed on the synthetic training dataset, designed to be sufficiently long so that repetition is unnecessary.
    - *Test data evaluation:* Conducted on the actual test dataset, with results averaged over 100 simulation runs for robustness.

In [ ]:
# default performance metrics
empty_metrics = {
    "profit": 0,
    "ave_quoted_price": 1,
    "ave_quoted_disutility": 1,
    "ave_payment": np.nan,
    "ave_realized_disutility": np.nan,
    "throughput": 0,
    "match_rate": np.nan,
    "cost_efficiency": np.nan,
    "ave_detour_rate_over_matched": np.nan,
}
metrics_list = empty_metrics.keys()

# evaluate the pricing policies on the training and test sets
# create the instance data for evaluation
eval_instance_data = InstanceData(
    c,
    sojourn_time,
    network,
    rider_types,
    arrival_rates,
    arrival_types,
    arrival_times,
    beta0,
    beta1,
    MAX_DETOUR_RATE,
)

# =============================================================================
# ======================= training data evaluation =============================
# =============================================================================
training_metrics = defaultdict(defaultdict)

# create the simulator for evaluation
train_eval_simulator = Simulator(
    copy.deepcopy(eval_instance_data), sim_params, use_real_data=False
)
train_eval_simulator.evaluation_time = training_synthetic_eval_time

# training data evaluation
for policy_key in [ITERATIVE_KEY, STATIC_KEY, MATCH_KEY]:
    policy_params = opt_params[beta0, beta1][policy_key]

    metrics, _ = evaluator(
        policy_key,
        pricing_policy[policy_key],
        matching_policy[policy_key],
        train_eval_simulator,
        max_iter=EVAL_MAX_ITER,
        set_type=TRAINING_SET_KEY,
        verbose=False,
        shrinkage_factor=policy_params["shrinkage_factor"],
        momentum_coef=policy_params["momentum"],
        update_disutility_flag=policy_params["update_disutil_eval"],
    )
    for metric in metrics_list:
        if policy_params["update_disutil_eval"]:
            training_metrics[policy_key][metric] = metrics[-1][metric]
        else:
            training_metrics[policy_key][metric] = metrics[metric]
        # convert profit to per minute per mile
        if metric == "profit":
            training_metrics[policy_key][metric] *= 60 / 1609
        # convert throughput to per minute
        elif metric == "throughput":
            training_metrics[policy_key][metric] *= 60

# create a dataframe for the performance metrics on the training set
training_metrics_df = pd.DataFrame.from_dict(
    [
        training_metrics[ITERATIVE_KEY],
        training_metrics[STATIC_KEY],
        {"profit": fluid_profits[beta0, beta1] * 60 / 1609},
        training_metrics[MATCH_KEY],
    ],
).T.rename(columns={0: ITERATIVE_KEY, 1: STATIC_KEY, 2: "static_UB", 3: MATCH_KEY})
training_metrics_df["match vs iterative"] = (
    training_metrics_df[MATCH_KEY] - training_metrics_df[ITERATIVE_KEY]
) / training_metrics_df[ITERATIVE_KEY]
training_metrics_df["match vs static"] = (
    training_metrics_df[MATCH_KEY] - training_metrics_df[STATIC_KEY]
) / training_metrics_df[STATIC_KEY]

# =============================================================================
# ======================= test data evaluation ================================
# =============================================================================
test_metrics = defaultdict(defaultdict)

# create the simulator for evaluation
test_eval_simulator = Simulator(
    copy.deepcopy(eval_instance_data), sim_params, use_real_data=True
)

# evaluate trained policies on the test set
for policy_key in [ITERATIVE_KEY, STATIC_KEY, MATCH_KEY]:
    policy_params = opt_params[beta0, beta1][policy_key]

    metrics, _ = evaluator(
        policy_key,
        pricing_policy[policy_key],
        matching_policy[policy_key],
        test_eval_simulator,
        max_iter=EVAL_MAX_ITER,
        set_type=TEST_SET_KEY,
        verbose=False,
        n_repeats=n_repeats,
        shrinkage_factor=policy_params["shrinkage_factor"],
        momentum_coef=policy_params["momentum"],
        update_disutility_flag=policy_params["update_disutil_eval"],
    )
    # average the performance metrics over the repetitions
    for metric in metrics_list:
        if policy_params["update_disutil_eval"]:
            test_metrics[policy_key][metric] = np.mean(
                [metrics[-1][n][metric] for n in range(n_repeats)]
            )
        else:
            test_metrics[policy_key][metric] = np.mean(
                [metrics[n][metric] for n in range(n_repeats)]
            )
        # convert profit to per minute per mile
        if metric == "profit":
            test_metrics[policy_key][metric] *= 60 / 1609
        # convert throughput to per minute
        elif metric == "throughput":
            test_metrics[policy_key][metric] *= 60

# create a dataframe for the performance metrics on the test set
test_metrics_df = pd.DataFrame.from_dict(test_metrics)
test_metrics_df["match vs iterative"] = (
    test_metrics_df[MATCH_KEY] - test_metrics_df[ITERATIVE_KEY]
) / test_metrics_df[ITERATIVE_KEY]
test_metrics_df["match vs static"] = (
    test_metrics_df[MATCH_KEY] - test_metrics_df[STATIC_KEY]
) / test_metrics_df[STATIC_KEY]

print("Training data performance metrics:")
display(training_metrics_df.round(3))
print("Test data performance metrics:")
display(test_metrics_df.round(3))

Training data performance metrics:


,iterative,static,static_UB,match,match vs iterative,match vs static
profit,0.917,1.183,1.312,1.599,0.743,0.352
ave_quoted_price,0.882,0.844,NaN,0.830,-0.059,-0.017
ave_quoted_disutility,0.913,0.876,NaN,0.859,-0.059,-0.019
ave_payment,0.860,0.817,NaN,0.787,-0.085,-0.036
ave_realized_disutility,0.890,0.852,NaN,0.822,-0.076,-0.035
throughput,2.465,3.509,NaN,3.974,0.612,0.132
match_rate,0.421,0.498,NaN,0.559,0.327,0.123
cost_efficiency,0.146,0.179,NaN,0.227,0.555,0.271
ave_detour_rate_over_matched,0.043,0.043,NaN,0.030,-0.290,-0.304


Test data performance metrics:


,iterative,static,match,match vs iterative,match vs static
profit,0.914,1.149,1.583,0.732,0.378
ave_quoted_price,0.885,0.848,0.833,-0.059,-0.018
ave_quoted_disutility,0.914,0.879,0.862,-0.056,-0.019
ave_payment,0.863,0.819,0.788,-0.086,-0.038
ave_realized_disutility,0.892,0.854,0.824,-0.076,-0.035
throughput,2.452,3.473,3.962,0.616,0.141
match_rate,0.416,0.489,0.561,0.349,0.148
cost_efficiency,0.142,0.174,0.224,0.580,0.290
ave_detour_rate_over_matched,0.044,0.043,0.031,-0.286,-0.282


### 